In [1]:
%matplotlib inline

def plot_pred():
    func = toolbox.compile(expr=hof[0])
    y_hat = []
    y = []
    for a, i in enumerate(t):
        y_hat.append(func(i))
        y.append(p[a])

    from matplotlib import pyplot as plt
    plt.plot(t,y)
    plt.plot(t, y_hat)
    plt.xlabel('time')  # x-axis
    plt.ylabel('x(t)')  # y-axis
    plt.title('Data')  # title
    plt.grid()  # grid
    plt.legend(['true','predicted'])
    plt.show()  # plot show

In [2]:
# -*- coding: utf-8 -*-
# 그 다음부터는 이거 여러번 실행해도 됨
"""
Created on Thu Jan  4 14:17:05 2018
@author: Bany
"""
###############################import###############################
import operator
import math
import random
import pandas as pd
import numpy
import time

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
import matplotlib.pyplot as plt



###################################################################

###################### Define functions############################
def safeDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
    
def sqrt(left):
    if left>=0:
        return left**0.5
    else:
        return 0
    
random.seed(time.time())
ran1 = int(random.random()*10)
ran2 =int(random.random()*10)

    
pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
#pset.addPrimitive(safeDiv, 2)
pset.addPrimitive(sqrt,1)
#pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand", lambda: random.randint(1,2))
#pset.addEphemeralConstant("rand"+str(ran1)+str(ran2), lambda: random.randint(min(ran1,ran2),max(ran1,ran2)))
pset.renameArguments(ARG0='x')



creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

#########################input DATA#################################

'''g = pd.read_csv('os.csv')
p = g['x']
t = list(numpy.arange(-2,2,0.01))
f = {}
for i in range(len(t)):
    f[t[i]] = p[i]'''

g = pd.read_csv('data.csv')
p = g['x']
t = list(numpy.arange(0,1000))
f = {}
for i in range(len(t)):
    f[t[i]] = p[i]

###################################################################



In [3]:
def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    start1 = time.time()
    # Evaluate the mean squared error between the expression
    sqerrors = ((func(x) - f[x])**2 for x in points)
    start2 = time.time()
    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),

###################################################################



In [ ]:

toolbox.register("evaluate", evalSymbReg, points=t)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))

## random.seed(318)

pop = toolbox.population(n=2000)
hof = tools.HallOfFame(1)

stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)
#############################################C.O, Muta,Gener##########
try:
    pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.3, 5000, stats=mstats,
                                   halloffame=hof, verbose=True)
    print (hof[0])
    plot_pred()
except MemoryError:
    print (hof[0])
    plot_pred()
except KeyboardInterrupt:
    print (hof[0])
    plot_pred()

    

# print log
#print (pop)
#print (log)



   	      	                      fitness                      	              size             
   	      	---------------------------------------------------	-------------------------------
gen	nevals	avg        	max        	min    	std        	avg  	max	min	std    
0  	2000  	4.90553e+20	9.80163e+23	1.01015	2.19116e+22	4.681	15 	2  	2.72493
1  	1727  	9.50837e+11	9.35032e+14	1.01015	2.95242e+13	4.401	16 	1  	2.55288
2  	1717  	4.94031e+20	9.88062e+23	1.01015	2.20882e+22	5.2815	24 	1  	2.98534
3  	1716  	4.95022e+20	9.90043e+23	1.01015	2.21325e+22	6.0915	26 	1  	3.41952
4  	1737  	9.90054e+20	1.98009e+24	1.01015	4.4265e+22 	6.9035	26 	1  	3.86422
5  	1673  	5.54069e+15	3.98402e+18	1.01015	1.31517e+17	7.852 	26 	1  	4.2516 
6  	1752  	1.0903e+22 	1.58724e+25	1.01015	3.6693e+23 	8.744 	29 	1  	4.6815 
7  	1741  	2.47909e+21	3.96811e+24	1.01015	9.14161e+22	9.457 	40 	1  	4.95148
8  	1730  	2.59671e+27	4.03786e+30	1.01015	9.29749e+28	10.5175	35 	1  	5.5837 
9  	1679  	3.39324e+21	3.96089e+

<string>:1: RuntimeWarning: overflow encountered in long_scalars


In [5]:
random.randint(1,100)/10

2.6

In [6]:
print(pop[60])

mul(sqrt(sub(mul(sqrt(2), sqrt(2)), 2)), mul(sqrt(x), sub(sub(x, sub(1, x)), sub(sin(x), x))))
